Karla Jacobo
DSCI 8950
EDA Assignment

To start off the Exploratory Data Analysis, I am going to import the data from my team's selected data sources and store them in local variables for easier access. 

In [ ]:
import pandas as pd

airline = pd.read_csv('../data/openFlightsRaw/airline.csv')
airports = pd.read_csv('../data/openFlightsRaw/airports.csv')
countries = pd.read_csv('../data/openFlightsRaw/countries.csv')
planes = pd.read_csv('../data/openFlightsRaw/planes.csv')
routes = pd.read_csv('../data/openFlightsRaw/routes.csv')